# Macroeconomic forecasting: Can machine learning methods outperform traditional approaches?

## Setup of the notebook


### Loading packages and modules

In [1]:
import os
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 250)

# models
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Lasso
from sklearn.ensemble import GradientBoostingRegressor 

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.impute import KNNImputer

from sklearn.model_selection import GridSearchCV

from pmdarima.arima import auto_arima

# pytorch
from torch import nn, no_grad, save, load
from torch import from_numpy, zeros
from torch.optim import SGD, Adam
from torch.utils.data import DataLoader, TensorDataset

import torch as T
import torch.nn as nn
import torch.nn.functional as F

# plots
import matplotlib.pyplot as plt
plt.style.use('seaborn-dark')

import seaborn as sns
sns.set_style("darkgrid")

import pickle

C:\Users\hauer\anaconda3\envs\test\lib\site-packages\sklearn\externals\six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
C:\Users\hauer\anaconda3\envs\test\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


# 1. Data

## 1.1 Real gross domestic product 

The gross domestic product (GDP) is the variable of interest.

Source of the data public availabe on the website of the IMF [here](https://www.imf.org/en/Publications/WEO/weo-database/2020/October/download-entire-database) and provided via an Excel file called `WEOApr2020all.xls`.

In [2]:
file = r"C:\Users\hauer\Dropbox\CFDS\Project\data2\WEOApr2020all.csv"
df_weo_real_gdp = pd.read_csv(file)

There are several types of data in this file. This is the description of the relevant subject, the growth of the GDP:

In [3]:
idx = df_weo_real_gdp['Subject Descriptor'] == 'Gross domestic product, constant prices'
df_weo_real_gdp['Subject Notes'] 

df_weo_real_gdp.loc[idx, 'Subject Notes'].unique()

array(['Annual percentages of constant price GDP are year-on-year changes; the base year is country-specific . Expenditure-based GDP is total final expenditures at purchasers? prices (including the f.o.b. value of exports of goods and services), less the f.o.b. value of imports of goods and services. [SNA 1993]'],
      dtype=object)

In [4]:
df_weo_real_gdp.loc[idx, 'Units'].unique()

array(['Percent change'], dtype=object)

The subject code given by the IMF is 'NGDP_RPCH'. This code will also occour in for the weo prediction data. 

In [5]:
df_weo_real_gdp.loc[idx, 'WEO Subject Code'].unique()

array(['NGDP_RPCH'], dtype=object)

As the data for the years are present in the columns of the dataframe, i exctract the relevant information and transpose it afterwars. I want the years as the rows and the variables as the columns. 

In [6]:
df_weo_real_gdp.columns

Index(['WEO Country Code', 'ISO', 'WEO Subject Code', 'Country',
       'Subject Descriptor', 'Subject Notes', 'Units', 'Scale',
       'Country/Series-specific Notes', '1980', '1981', '1982', '1983', '1984',
       '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993',
       '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002',
       '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011',
       '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020',
       '2021', 'Estimates Start After'],
      dtype='object')

This is done with the function: 

In [7]:
def get_imf_woe_data(df_weo_real_gdp, country, remove_na=False):

    df = df_weo_real_gdp[df_weo_real_gdp['Country'] == country]
    
    result = pd.DataFrame()
    
    available_variables = df['Subject Descriptor'].unique()
    
    for variable in available_variables:
        df_curr = df[df['Subject Descriptor'] == variable]
        df_curr = df_curr.iloc[:, 9:49]
        df_curr = df_curr.transpose()
        df_curr = df_curr.rename({df_curr.columns[0]: variable}, axis='columns')
        result = pd.concat([result, df_curr], axis=1)
        
        
    if remove_na:
        result = result.dropna(axis=1) 
        
    return result

To extract the GDP data i use this function: 

In [8]:
def get_gdp_real(df_weo_real_gdp, country):
    df = get_imf_woe_data(df_weo_real_gdp, country, remove_na=False)
    df.index = df.index.astype(dtype='int64')   
    df['GDP real'] = df['Gross domestic product, constant prices']  
    df['GDP real'] = df['GDP real'].str.replace(',', '').astype('float')
    return df['GDP real']

Here is for example the real GDP growth for germany:

In [9]:
get_gdp_real(df_weo_real_gdp, 'Germany')

1980    1.272
1981    0.110
1982   -0.788
1983    1.555
1984    2.826
1985    2.192
1986    2.417
1987    1.469
1988    3.736
1989    3.913
1990    5.723
1991    5.011
1992    1.925
1993   -0.976
1994    2.395
1995    1.541
1996    0.814
1997    1.790
1998    2.019
1999    1.885
2000    2.905
2001    1.689
2002   -0.201
2003   -0.708
2004    1.186
2005    0.728
2006    3.815
2007    2.975
2008    0.965
2009   -5.694
2010    4.185
2011    3.913
2012    0.428
2013    0.431
2014    2.218
2015    1.742
2016    2.230
2017    2.465
2018    1.522
2019    0.565
Name: GDP real, dtype: float64

The real GDP is availabe for the following 195 (one occurrence is Nan) countries: 

In [10]:
countries_woe_real = df_weo_real_gdp['Country'].unique()
len(countries_woe_real)

195

In [11]:
countries_woe_real

array(['Afghanistan', 'Albania', 'Algeria', 'Angola',
       'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba',
       'Australia', 'Austria', 'Azerbaijan', 'The Bahamas', 'Bahrain',
       'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin',
       'Bhutan', 'Bolivia', 'Bosnia and Herzegovina', 'Botswana',
       'Brazil', 'Brunei Darussalam', 'Bulgaria', 'Burkina Faso',
       'Burundi', 'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada',
       'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia',
       'Comoros', 'Democratic Republic of the Congo', 'Republic of Congo',
       'Costa Rica', "Côte d'Ivoire", 'Croatia', 'Cyprus',
       'Czech Republic', 'Denmark', 'Djibouti', 'Dominica',
       'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador',
       'Equatorial Guinea', 'Eritrea', 'Estonia', 'Eswatini', 'Ethiopia',
       'Fiji', 'Finland', 'France', 'Gabon', 'The Gambia', 'Georgia',
       'Germany', 'Ghana', 'Greece', 'Grenada', 'Guatemala', 'Gui

##  The variables 

### IMF

The IMF proivdes some economic variables along with the realised GDP, that are already in the `df_weo_real_gdp` dataframe:

In [12]:
df_imf_woe_data =  get_imf_woe_data(df_weo_real_gdp, 'Germany', remove_na=False)
df_imf_woe_data.head()

,"Gross domestic product, constant prices","Gross domestic product, current prices","Gross domestic product per capita, constant prices","Inflation, average consumer prices","Inflation, end of period consumer prices",Unemployment rate,General government net lending/borrowing,Current account balance
1980,1.272,867.363,0.931,5.447,NaN,3.359,NaN,-1.782
1981,0.11,950.471,-0.078,6.324,NaN,4.831,NaN,-0.684
1982,-0.788,"1,001.24",-0.717,5.256,NaN,6.734,NaN,0.866
1983,1.555,"1,056.63",1.91,3.284,NaN,8.099,NaN,0.666
1984,2.826,"1,125.70",3.243,2.396,NaN,8.058,NaN,1.423


I will use the following quantities:

In [13]:
imf_woe_variables = ['Inflation, average consumer prices', 'Unemployment rate', 'General government net lending/borrowing', 'Current account balance']

for x in imf_woe_variables:
    idx = df_weo_real_gdp['Subject Descriptor'] == x
    print(x)
    print(df_weo_real_gdp.loc[idx, 'Subject Notes'].unique())
    print(df_weo_real_gdp.loc[idx, 'Units'].unique())
    print('')

Inflation, average consumer prices
['Annual percentages of average consumer prices are year-on-year changes.']
['Percent change']

Unemployment rate
['Unemployment rate can be defined by either the national definition, the ILO harmonized definition, or the OECD harmonized definition. The OECD harmonized unemployment rate gives the number of unemployed persons as a percentage of the labor force (the total number of people employed plus unemployed). [OECD Main Economic Indicators, OECD, monthly] As defined by the International Labour Organization, unemployed workers are those who are currently not working but are willing and able to work for pay, currently available to work, and have actively searched for work. [ILO, http://www.ilo.org/public/english/bureau/stat/res/index.htm]']
['Percent of total labor force']

General government net lending/borrowing
['Net lending (+)/ borrowing (?) is calculated as revenue minus total expenditure. This is a core GFS balance that measures the extent to

So only `Inflation, average consumer prices` is given by an annual percentage change, the other variables needs to be transformed later. 

The dataframe for example for germany will look like this:

In [14]:
df_imf_woe_data =  get_imf_woe_data(df_weo_real_gdp, 'Germany', remove_na=False)
df_imf_woe_data[imf_woe_variables]

,"Inflation, average consumer prices",Unemployment rate,General government net lending/borrowing,Current account balance
1980,5.447,3.359,NaN,-1.782
1981,6.324,4.831,NaN,-0.684
1982,5.256,6.734,NaN,0.866
1983,3.284,8.099,NaN,0.666
1984,2.396,8.058,NaN,1.423
1985,2.084,8.124,NaN,2.662
1986,-0.125,7.834,NaN,4.024
1987,0.242,7.843,NaN,3.709
1988,1.274,7.735,NaN,4.317
1989,2.778,6.79,NaN,4.689


###  OECD

The Organisation for Economic Co-operation and Development (OECD) provied also macroeconomic data in its [iLibrary]( https://www.oecd-ilibrary.org). The indicators can be browsed by theme and I choose 37 to use for my forecast. Each one is available by an `.csv` file. I load all of them together into one dataframe: 

In [15]:
def get_oecd_data(path_oecd, country): 
 
    result = pd.DataFrame()
  
    
    for file_name in os.listdir(path_oecd):
    
        
        file = os.path.join(path_oecd, file_name)
        
        df_orig = pd.read_csv(file)
        unique_subjects = df_orig['SUBJECT'].unique()
    
        
        for subject in unique_subjects:
            
            
            df = df_orig.copy()
            df = df[df['LOCATION'] == country] 
            df = df[df['SUBJECT'] == subject]
            
            # if there is only one unique subject, the name is TOT
            if(len(unique_subjects) == 1):
                subject = file_name[:-4]
            
            
            df = df.rename({df.columns[6]: subject}, axis='columns')
            
            df = df.set_index('TIME')
            
            result = pd.concat([result, df[subject]], axis=1)           
    
    return result

In [16]:
path_oecd = r'C:\Users\hauer\Dropbox\CFDS\Project\data2\OECD'

df_oecd = get_oecd_data(path_oecd, 'USA')
df_oecd.tail()

,Air_Pollution,Crude_oil_production,CurrentAccountBalance,Electricity_generation,ExchangeR,Fertility_rates,GHG,Household_spending,NOMINAL,RENT,...,STINT,Tax_on_corporate_profits,Tax_on_goods_and_services,Tax_on_personal_income,TermsOfTrade,TradeGoodsExport,TradeGoodsImport,TradeServicesExport,TradeServicesImports,Working_age_population
2016,7.37633,446538.263,-2.109880,4119445.000,1.0,1.8,14.9,12769962.0,105.711763,103.772816,...,0.644167,1.949,4.299,10.387,101.649388,1451.022,2187.599,780531.0,511898.0,65.890
2017,7.36365,470360.949,-1.871310,4086002.602,1.0,1.8,14.6,13340354.0,112.336757,107.730991,...,1.152500,1.566,4.223,10.422,101.989233,1546.472,2339.885,830387.0,544836.0,65.628
2018,NaN,NaN,-2.185078,4236927.396,1.0,1.7,NaN,13993281.0,119.426500,111.627442,...,2.188333,0.998,4.303,10.037,102.492082,1665.688,2537.730,862434.0,562069.0,65.364
2019,NaN,NaN,-2.241142,NaN,1.0,NaN,NaN,14544601.0,125.599526,115.765067,...,NaN,0.957,4.296,10.143,NaN,1643.161,2497.532,875825.0,588359.0,NaN
2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


All of the variables are given absolute values for the specific year, so every variable needs to be transformed afterwards.
There are 182 countries or aggregated country groups available:

In [17]:
countries_oecd = set()

for file_name in os.listdir(path_oecd):
    file = os.path.join(path_oecd, file_name)
    df = pd.read_csv(file)
    countries_current = set(df['LOCATION'].unique())
    
    countries_oecd = countries_oecd.union(countries_current)

    
countries_oecd = list(countries_oecd)
len(countries_oecd)

183

In [18]:
countries_oecd

['MAR',
 'TWN',
 'PNG',
 'OAVG',
 'EU',
 'BHR',
 'KGZ',
 'G20',
 'EA19',
 'GHA',
 'ZWE',
 'PAN',
 'AGO',
 'SWE',
 'BFA',
 'JOR',
 'HRV',
 'KAZ',
 'RWA',
 'LBR',
 'KWT',
 'CAF',
 'ETH',
 'NZL',
 'LSO',
 'CIV',
 'TUN',
 'VNM',
 'ALB',
 'BEL',
 'PAK',
 'CUB',
 'TCD',
 'PRK',
 'KHM',
 'UGA',
 'TTO',
 'TGO',
 'EA17',
 'NPL',
 'EU28',
 'THA',
 'ROU',
 'MEX',
 'DOM',
 'SGP',
 'DEW',
 'PRT',
 'POL',
 'BOL',
 'ESP',
 'BDI',
 'COD',
 'COG',
 'SDN',
 'DZA',
 'TUR',
 'QAT',
 'ZMB',
 'LBN',
 'MDG',
 'AFG',
 'EST',
 'USA',
 'SEN',
 'NER',
 'G-20',
 'G7M',
 'KOR',
 'BEN',
 'MKD',
 'PER',
 'EU27_2020',
 'CRI',
 'EA',
 'GBR',
 'SAU',
 'SLE',
 'MYS',
 'BRN',
 'URY',
 'LIE',
 'OEU',
 'GNQ',
 'LKA',
 'GIN',
 'SWZ',
 'BTN',
 'GNB',
 'ERI',
 'IRL',
 'ISL',
 'PSE',
 'AZE',
 'KEN',
 'DNK',
 'DEU',
 'ZAF',
 'MOZ',
 'RUS',
 'VEN',
 'SRB',
 'MNG',
 'SYR',
 'ARG',
 'WLD',
 'MDA',
 'BGD',
 'SLV',
 'UZB',
 'JPN',
 'IDN',
 'MAC',
 'NIC',
 'HND',
 'FIN',
 'LTU',
 'MMR',
 'ISR',
 'AUT',
 'HUN',
 'MLT',
 'TLS',
 'CHE',

Here is the mapping from the ISO country code that used the OECD to the country names that are used by the IMF:

In [19]:
path = r'C:\Users\hauer\Dropbox\CFDS\Project\data2\Mapping_country_codes.csv'
df_country_mapping =  pd.read_csv(path, sep = '\t')

df_country_mapping

,ID,ISO,Country
0,1,AFG,Afghanistan
1,2,ALB,Albania
2,3,DZA,Algeria
3,4,AGO,Angola
4,5,ATG,Antigua and Barbuda
5,6,ARG,Argentina
6,7,ARM,Armenia
7,8,ABW,Aruba
8,9,AUS,Australia
9,10,AUT,Austria


##   World Economic Outlook 


The International Monetary Fund publishes predictions of the GDP growth in its World Economic Outlook. The data can is taken from [here](https://www.imf.org/en/Publications/WEO/weo-database/2020/October) in the related links Historical WEO Forecasts Database. The data is provided in an Excel file called `WEOhistorical.xlsx`. The IMF publishes the WEO twice a year in spring and in fall. I will use the prediction of the fall, as this closer to the next year and therefore the prediction should be more precise. The data is formated the following: 

| country | year   |F1990ngdp_rpch|
|------|--------|--------------|
|   germany  | 1988  | 4.08 |
|   germany  | 1989  | 2.96 |
|   germany  | 1990  | 1.98 |
|   germany  | 1991  | 2.44 |
|   germany  | 1992  | 3.42 |
|   germany  | 1993  | 3.45 |
|   germany  | 1994  | 3.42 |
|   germany  | 1995  | 3.40 |

This is for example the WEO in fall of 1990 for germany. There are two years of historical data and 6 years of forecast data. The forecast can be found in the column `F1990ngdp_rpch`. This is the same subject code as for the realised GDP. I will only use the forecast for the next year, so for 1990 if will use the predicted growth of the GDP in 1991. I extract the forecast for a certain country and prediciton horizon with the following function. First I load the Excel file into an pandas dataframe:

In [20]:
path = r'C:\Users\hauer\Dropbox\CFDS\Project\data2\WEOhistorical.xlsx'
df_weo =  pd.read_excel(path,sheet_name='ngdp_rpch')

The function for the extraction of the WEO is called `get_predictions_weo`:

In [21]:
def get_predictions_weo(df_weo, country, start_forecast, end_forecast):
       
    df = df_weo[df_weo['country'] == country]
    
    
    for col in df.columns:
        if 'S' in col:
            del df[col] 
            
    del df['WEO_Country_Code']     
    
    
    df = df[df['year'] >= start_forecast]
    
    
    predictions_weo = []
    years = np.arange(start_forecast, end_forecast+1, 1)
    
    for year in years:
       
        df_curr = df[df['year'] == year]
        
        year_WEO = year - 1 
        column = 'F' + str(year_WEO) + 'ngdp_rpch'
        y_pred_year = df_curr[column].values[0]
        
        predictions_weo.append(y_pred_year)
    
    predictions_weo = pd.Series(data = predictions_weo, index = years)
    
    return predictions_weo

Here is for example the WEO for germany for the years 2010 to 2018:


In [22]:
get_predictions_weo(df_weo, country = 'Germany', start_forecast =  2010, end_forecast = 2018)

2010    0.335834
2011    2.021567
2012    1.273123
2013    0.852179
2014    1.399657
2015    1.451330
2016    1.573023
2017    1.425094
2018    1.843451
dtype: float64

The WEO is available from 1980 for the following countries or aggregated country groups:

In [23]:
countries_woe = df_weo['country'].unique()
len(countries_woe)

199

In [24]:
countries_woe

array(['World', 'Advanced Economies', 'United States', 'United Kingdom',
       'Austria', 'Belgium', 'Denmark', 'France', 'Germany', 'San Marino',
       'Italy', 'Luxembourg', 'Netherlands', 'Norway', 'Sweden',
       'Switzerland', 'Canada', 'Japan', 'Euro area', 'Finland', 'Greece',
       'Iceland', 'Ireland', 'Malta', 'Portugal', 'Spain', 'Turkey',
       'Australia', 'New Zealand', 'South Africa',
       'Emerging Market and Developing Economies', 'Argentina', 'Bolivia',
       'Brazil', 'Chile', 'Colombia', 'Costa Rica', 'Dominican Republic',
       'Ecuador', 'El Salvador', 'Guatemala', 'Haiti', 'Honduras',
       'Mexico', 'Nicaragua', 'Panama', 'Paraguay', 'Peru', 'Uruguay',
       'Venezuela', 'Antigua and Barbuda', 'Bahamas, The', 'Aruba',
       'Barbados', 'Dominica', 'Grenada', 'Guyana', 'Belize', 'Jamaica',
       'Puerto Rico', 'St. Kitts and Nevis', 'St. Lucia',
       'St. Vincent and the Grenadines', 'Suriname',
       'Trinidad and Tobago', 'Bahrain', 'Cyprus', 'I

#  Joinig the datasets


Here I join the different data sets to get a dataframe for each country. The data is available from 1980 to 2019 expect for the OECD data sets. These are provided from 1970 to 2017 and hence I filter them to receive the time from 1980 to 2017. I use the pythonic try except block to select only countries, that have an correspondent ISO code in the OECD dataset.
I will save each individual dataframe in a dictionary. For later convenience I also rename the column of the real gdp.

In [25]:
database = {}

availabe_countries_woe = set(countries_woe_real).union(countries_woe)

for country in availabe_countries_woe:
    try:
        iso = df_country_mapping[df_country_mapping['Country'] == country]['ISO']
        iso = iso.values[0]
        
        df_real_gdp = get_gdp_real(df_weo_real_gdp, country)
        df_real_gdp = df_real_gdp[df_real_gdp.index <= 2017]
        
        df_imf_woe_data =  get_imf_woe_data(df_weo_real_gdp, country, remove_na=False)
        df_imf_woe_data = df_imf_woe_data[imf_woe_variables]
        df_imf_woe_data.index = df_imf_woe_data.index.astype(int)
        df_imf_woe_data = df_imf_woe_data[df_imf_woe_data.index <= 2017]
        
        df_oecd = get_oecd_data(path_oecd, iso)
        df_oecd = df_oecd[df_oecd.index >= 1980]
        df_oecd = df_oecd[df_oecd.index <= 2017]
        
        
        df = pd.concat([df_real_gdp, df_imf_woe_data, df_oecd], axis=1)
        
        df = df.rename(columns={"GDP real": "y"})
        
        database[country] = df

    except Exception as e:
        print('Error' + str(e) + ' for ' + str(country))

Errorindex 0 is out of bounds for axis 0 with size 0 for nan
Errorindex 0 is out of bounds for axis 0 with size 0 for Emerging Market and Developing Economies
Errorcould not convert string to float: '--' for Ethiopia
Errorcould not convert string to float: '--' for Gabon
Errorcould not convert string to float: '--' for Angola
Errorindex 0 is out of bounds for axis 0 with size 0 for Iran
Errorcould not convert string to float: '--' for Greece
Errorindex 0 is out of bounds for axis 0 with size 0 for Euro area
Errorcould not convert string to float: '--' for Mauritania
Errorindex 0 is out of bounds for axis 0 with size 0 for Advanced Economies
Errorindex 0 is out of bounds for axis 0 with size 0 for Congo, Republic of
Errorindex 0 is out of bounds for axis 0 with size 0 for World
Errorindex 0 is out of bounds for axis 0 with size 0 for West Bank and Gaza
Errorindex 0 is out of bounds for axis 0 with size 0 for Montenegro, Rep. of
Errorindex 0 is out of bounds for axis 0 with size 0 for Ba

Now I have 189 dataframes that are ready to be analysed: 

In [26]:
len(database.keys())

189

## Fixing data type

During the later analysis there occured problems with wrong datatypes. For example some vaules where saved as strings: `unsupported operand type(s) for /: 'str' and 'str'`. In this section I fix all kind of this problems. 

Here is a list of the issues which i fix in the same order in the following code: 
* Netherlands could not convert string to float: '--'
* Brazil could not convert string to float: '1,430.72'
* Hungary unsupported operand type(s) for /: 'str' and 'str'


In [27]:
def convert_string_series(x):
    try:
        return x.str.replace(',', '').astype(float)
    
    # this operation only works with string vaules, i was not able to filter the other dtypes
    # (that should only be float) but it did not work. 
    
    except:
        return x


database_fixed = {}


for country in database.keys():
    try: 
        # Netherlands could not convert string to float: '--'
        database_fixed[country] =  database[country].replace('--', np.nan)

        # Brazil could not convert string to float: '1,430.72'
        database_fixed[country] =  database[country].replace(',', '')

        # Hungary unsupported operand type(s) for /: 'str' and 'str'
        database_fixed[country]  = database[country].apply(convert_string_series)
        
       
    except Exception as e:
        print(country + " " + str(e))


database = database_fixed.copy()

## Filitering missing values


In [28]:
variables = database['France'].columns
missing_dict = {var:0 for var in variables}

In [29]:
number_of_observations = 0
for country in database.keys():
    df =  database[country]   
    number_of_observations += df.shape[0]

    for column in df.columns:
        column_current = df[column]
        number_of_missing_observations = sum(column_current.isnull())
        missing_dict[column] += number_of_missing_observations   

In [30]:
df_missing = pd.DataFrame.from_dict(missing_dict, orient='index')
df_missing.columns = ['Number of missing entries']
df_missing['Percent of missing entries'] = df_missing['Number of missing entries'] / number_of_observations * 100
df_missing.sort_values(by='Percent of missing entries', ascending=1)

,Number of missing entries,Percent of missing entries
y,794,11.055416
"Inflation, average consumer prices",819,11.403509
Current account balance,1105,15.385686
Material_consumption,2142,29.824561
TOTMAT,2233,31.091618
General government net lending/borrowing,2309,32.149819
Crude_oil_production,2493,34.711779
GHG,2517,35.045948
NONNRGMAT,3431,47.772208
Unemployment rate,3764,52.408800


How many missing values are there for each country in total:

In [31]:
df_missing_countries = pd.DataFrame(columns=['country', 'Percent of missing entries'])
   


for country in database.keys():
    df =  database[country] 
    N, p = df.shape
    number_of_entries = N * p
    number_of_missing = df.isna().sum().sum()
    percent_of_missing = number_of_missing / number_of_entries
    
    df_missing_countries = pd.concat([pd.DataFrame([[country , percent_of_missing]],
                                                        columns=df_missing_countries.columns),
                                         df_missing_countries],ignore_index=True)


df_missing_countries.sort_values(by='Percent of missing entries', ascending=1)


,country,Percent of missing entries
164,United States,0.053258
7,Canada,0.055138
185,Australia,0.062030
62,New Zealand,0.073308
44,United Kingdom,0.102757
49,France,0.111529
56,Sweden,0.117168
132,Germany,0.117794
39,Spain,0.119048
86,Switzerland,0.122807


Filtering out countries that have more than 50 % missing values: 

In [32]:
database_filtered_countries = {}


for country in database.keys():
    df =  database[country] 
    N, p = df.shape
    number_of_entries = N * p
    number_of_missing = df.isna().sum().sum()
    percent_of_missing = number_of_missing / number_of_entries
    
    if percent_of_missing >= 0.50:
        continue
    
    database_filtered_countries[country] = df
len(database_filtered_countries.keys())

58

Next i want to analyse how many values are missing. So I can decide which of the variables I will use for the models. To get an overview I count the number of missing values for all variables of all countries. To do this, I create a dictionary with all varibales available as keys. Then I only continue with the top 15 columns that are filed with values: 
    
    

In [33]:
variables = database_filtered_countries['France'].columns
missing_dict = {var:0 for var in variables}

number_of_observations = 0
for country in database_filtered_countries.keys():
    df =  database_filtered_countries[country]   
    number_of_observations += df.shape[0]

    for column in df.columns:
        column_current = df[column]
        number_of_missing_observations = sum(column_current.isnull())
        missing_dict[column] += number_of_missing_observations 
        
df_missing = pd.DataFrame.from_dict(missing_dict, orient='index')
df_missing.columns = ['Number of missing entries']
df_missing['Percent of missing entries'] = df_missing['Number of missing entries'] / number_of_observations * 100
df_missing.sort_values(by='Percent of missing entries', ascending=1)

,Number of missing entries,Percent of missing entries
GHG,56,2.540835
Crude_oil_production,60,2.722323
y,134,6.079855
"Inflation, average consumer prices",135,6.125227
Current account balance,151,6.851180
ExchangeR,253,11.479129
Population,266,12.068966
Working_age_population,266,12.068966
Fertility_rates,268,12.159710
Unemployment rate,274,12.431942


Selecting the top 15 columns:

In [34]:
df_missing = df_missing.sort_values(by='Percent of missing entries', ascending=1)

selected_columns = df_missing.index[0:15]
selected_columns.values

array(['GHG', 'Crude_oil_production', 'y',
       'Inflation, average consumer prices', 'Current account balance',
       'ExchangeR', 'Population', 'Working_age_population',
       'Fertility_rates', 'Unemployment rate',
       'Patents_on_environment_technologies', 'PPP', 'TOTMAT',
       'Material_consumption', 'General government net lending/borrowing'],
      dtype=object)

Ordering the columns new, because y has to be in first place and create a new database: 

In [35]:
columns = ['y','GHG', 'Crude_oil_production',
       'Inflation, average consumer prices', 'Current account balance',
       'ExchangeR', 'Population', 'Working_age_population',
       'Fertility_rates', 'Unemployment rate',
       'Patents_on_environment_technologies', 'PPP', 'TOTMAT',
       'Material_consumption', 'General government net lending/borrowing']

database_filtered_columns = {}

for country in database_filtered_countries.keys():
    df = database_filtered_countries[country]
    database_filtered_columns[country] = df[columns]


# Spliting the dataset

The reference for this section is the book "The Elements of Statistical Learning" from Hastie et. al. 

To obtain an accurate Data Science process, it is necessary to split the whole dataset in certain subsets.  This is important for two reasons:

* Model selection: estimating the performance of different models in order to choose the best one. The term model selection also includes the tuning of the hyperparameters, if you define a model as the tupel consisting of the data used for training, the concrete typ of model or algorithm and the hyperparameters of the later. 
* Model assessment: having chosen a final model, estimating its prediction error (generalization error) on new data.

The data is split into a training, validation and test set. The training set is used to fit the model and the validation set is used to calculate the validation error. This error gives an estimate of its prediction error. The test is kept in an "vault" and is brought out only at the very end of the analysis. After using the test set, no changes in any step is allowed. Otherwise the test error will underestimate the true test error. With this test error the selection of the best model is done. 

Another thing in this procedure is very important: Every step in the analysis needs to be performed on the training set only. For example in the next section I will impute missing values. I will fit the algorithim that performes the imputation on the training set and predict the values then for both the training and test set. Otherwise the wrong applicaton of the impuation would also underestimate the validation or test error, because for the imputation there would be more information available than in an real life scenario. New data is completly unseen and only the training data is available for fitting the model in generall. 

This is also noted in the section 7.10.2 The Wrong and Right Way to Do Cross-validation. Even though I am not doing cross validation, this description suits the application in this project. 


I will use the years 2014 - 2017 as the test set. I will calculate  the performance on the very end and after i will not change anythin in the procedure. The years  2010 - 2013 will be the validation set and will be use for tuning the hypterparameter. 

In [36]:
def split_dataset(database):
    database_training = {}
    database_validation = {}
    database_test = {}


    for country in database.keys():


        df = database[country]

        #df_test = df[df.index > 2013]
        #df_validation = df[(df.index > 2009) & (df.index <= 2013)]
        #df_training = df[df.index <= 2009]
        
#         df_test = df[df.index > 2012]
#         df_validation = df[(df.index > 2008) & (df.index <= 2012)]
#         df_training = df[df.index <= 2008]
        
        df_test = df[df.index > 2010]
        df_validation = df[(df.index > 2004) & (df.index <= 2010)]
        df_training = df[df.index <= 2004]

        database_training[country] = df_training
        database_validation[country] = df_validation
        database_test[country] = df_test
        
    return (database_training, database_validation, database_test)


database_training, database_validation, database_test = split_dataset(database_filtered_columns)

### kNN

Example for 'Italy'. knn performes as mean, if all values are NaN. 

In [37]:
database_training['Italy']

,y,GHG,Crude_oil_production,"Inflation, average consumer prices",Current account balance,ExchangeR,Population,Working_age_population,Fertility_rates,Unemployment rate,Patents_on_environment_technologies,PPP,TOTMAT,Material_consumption,General government net lending/borrowing
1980,3.099,6.3,1710.017,21.800,-3.518,0.442318,56.433876,64.578,1.7,7.370,4.77,0.381979,NaN,NaN,NaN
1981,0.557,6.1,1364.214,19.510,-3.532,0.587090,56.501674,65.117,1.6,7.649,5.20,0.414519,NaN,NaN,NaN
1982,0.156,6.0,1640.666,16.460,-2.375,0.698513,56.543539,65.789,1.6,8.288,5.55,0.458879,NaN,NaN,NaN
1983,0.925,5.9,2097.621,14.700,0.305,0.784420,56.564076,66.500,1.5,7.367,3.65,0.508248,NaN,NaN,NaN
1984,3.013,6.0,2128.021,10.740,-0.909,0.907395,56.576719,67.167,1.5,7.833,4.60,0.543343,NaN,NaN,NaN
1985,2.621,6.0,2234.422,8.969,-1.218,0.986143,56.593070,67.560,1.4,8.167,4.81,0.575008,NaN,NaN,NaN
1986,2.710,6.1,2402.574,5.820,0.426,0.769939,56.596157,67.786,1.4,8.867,4.65,0.605866,NaN,NaN,NaN
1987,3.067,6.4,3712.637,4.720,-0.359,0.669364,56.601936,68.022,1.4,9.617,4.48,0.626777,NaN,NaN,NaN
1988,4.029,6.5,4570.496,5.090,-0.894,0.672235,56.629290,68.216,1.4,9.683,4.40,0.645684,NaN,NaN,-10.543
1989,3.252,6.8,4349.144,6.245,-1.605,0.708627,56.671778,68.395,1.3,9.667,5.68,0.659832,NaN,NaN,-10.909


In [38]:
df = database_training['Italy']

imputer = KNNImputer(missing_values=np.nan, n_neighbors=5, weights="uniform")

imputer.fit(df)

df_data = imputer.transform(df)

df = pd.DataFrame(data = df_data, columns = df.columns, index = df.index)
df

,y,GHG,Crude_oil_production,"Inflation, average consumer prices",Current account balance,ExchangeR,Population,Working_age_population,Fertility_rates,Unemployment rate,Patents_on_environment_technologies,PPP,TOTMAT,Material_consumption,General government net lending/borrowing
1980,3.099,6.3,1710.017,21.800,-3.518,0.442318,56.433876,64.578,1.7,7.370,4.77,0.381979,2.77092,13.11344,-8.9812
1981,0.557,6.1,1364.214,19.510,-3.532,0.587090,56.501674,65.117,1.6,7.649,5.20,0.414519,2.77092,13.11344,-8.9812
1982,0.156,6.0,1640.666,16.460,-2.375,0.698513,56.543539,65.789,1.6,8.288,5.55,0.458879,2.77092,13.11344,-8.9812
1983,0.925,5.9,2097.621,14.700,0.305,0.784420,56.564076,66.500,1.5,7.367,3.65,0.508248,2.77092,13.11344,-8.9812
1984,3.013,6.0,2128.021,10.740,-0.909,0.907395,56.576719,67.167,1.5,7.833,4.60,0.543343,2.77092,13.11344,-8.9812
1985,2.621,6.0,2234.422,8.969,-1.218,0.986143,56.593070,67.560,1.4,8.167,4.81,0.575008,2.77092,13.11344,-8.9812
1986,2.710,6.1,2402.574,5.820,0.426,0.769939,56.596157,67.786,1.4,8.867,4.65,0.605866,2.77092,13.11344,-8.9812
1987,3.067,6.4,3712.637,4.720,-0.359,0.669364,56.601936,68.022,1.4,9.617,4.48,0.626777,2.77092,13.11344,-8.9812
1988,4.029,6.5,4570.496,5.090,-0.894,0.672235,56.629290,68.216,1.4,9.683,4.40,0.645684,2.80044,12.88398,-10.5430
1989,3.252,6.8,4349.144,6.245,-1.605,0.708627,56.671778,68.395,1.3,9.667,5.68,0.659832,2.75428,12.78984,-10.9090


For the validation and training set the `transform` function is also applied to impute missing values for these datapoints: 

In [39]:
df = database_validation['Italy']
df_data = imputer.transform(df)
df_validation = pd.DataFrame(data = df_data, columns = df.columns, index = df.index)
df_validation

,y,GHG,Crude_oil_production,"Inflation, average consumer prices",Current account balance,ExchangeR,Population,Working_age_population,Fertility_rates,Unemployment rate,Patents_on_environment_technologies,PPP,TOTMAT,Material_consumption,General government net lending/borrowing
2005,0.818,7.8,6109.832,2.207,-0.883,0.803800,57.969482,66.182,1.3,7.683,6.88,0.855139,2.8184,14.3245,-4.083
2006,1.791,7.7,5767.898,2.229,-1.476,0.796433,58.143980,65.907,1.4,6.792,7.58,0.823835,2.8100,14.5596,-3.617
2007,1.487,7.5,5858.880,2.025,-1.367,0.729672,58.438309,65.761,1.4,6.125,8.94,0.810223,2.9258,14.1416,-1.340
2008,-0.962,7.2,5219.003,3.492,-2.795,0.679923,58.826733,65.673,1.4,6.742,10.30,0.783694,3.0210,13.5238,-2.564
2009,-5.281,6.4,4550.130,0.765,-1.885,0.716958,59.095367,65.553,1.4,7.733,11.72,0.770581,3.2094,12.0204,-5.121
2010,1.713,6.6,5079.029,1.620,-3.295,0.754309,59.277414,65.459,1.4,8.350,11.01,0.772612,3.3746,11.5846,-4.240


In [40]:
df = database_test['Italy']
df_data = imputer.transform(df)
df_test = pd.DataFrame(data = df_data, columns = df.columns, index = df.index)
df_test

,y,GHG,Crude_oil_production,"Inflation, average consumer prices",Current account balance,ExchangeR,Population,Working_age_population,Fertility_rates,Unemployment rate,Patents_on_environment_technologies,PPP,TOTMAT,Material_consumption,General government net lending/borrowing
2011,0.707,6.4,5282.990,2.935,-2.825,0.718414,59.379446,65.291,1.4,8.392,11.06,0.758687,3.3574,11.6746,-3.593
2012,-2.981,6.1,5395.969,3.315,-0.231,0.778338,59.539725,64.994,1.4,10.683,11.71,0.747731,3.8045,9.9469,-2.945
2013,-1.841,5.6,5500.949,1.245,1.109,0.752945,60.233944,64.761,1.4,12.133,10.53,0.737299,4.4085,8.3859,-2.854
2014,-0.005,5.3,5763.898,0.234,1.899,0.752728,60.789144,64.578,1.4,12.608,10.26,0.739644,4.4308,8.3097,-2.954
2015,0.778,5.4,5470.000,0.108,1.421,0.901296,60.730585,64.388,1.4,11.917,10.15,0.738495,4.6079,8.0301,-2.552
2016,1.293,5.4,3746.000,-0.050,2.594,0.903421,60.627494,64.238,1.3,11.667,9.90,0.700615,4.6541,8.0420,-2.404
2017,1.668,5.3,4098.000,1.326,2.575,0.885206,60.536713,64.124,1.3,11.267,10.15,0.686535,4.7564,7.9989,-2.445


Finally this step is done for the whole dataset: 

In [41]:
database_training_imputed = {}
database_validation_imputed = {}
database_test_imputed = {}

imputer = KNNImputer(missing_values=np.nan, n_neighbors=5, weights="uniform")

for country in database_filtered_columns.keys():   
  
    try:
        
        df_current = database_training[country]
        imputer.fit(df_current)
        df_data = imputer.transform(df_current)
        database_training_imputed[country] = pd.DataFrame(data = df_data,
                                                          columns = df_current.columns,
                                                          index = df_current.index)
        
        df_current = database_validation[country]
        df_data = imputer.transform(df_current)
        database_validation_imputed[country] = pd.DataFrame(data = df_data,
                                                            columns = df_current.columns,
                                                            index = df_current.index)
        
        df_current = database_test[country]
        df_data = imputer.transform(df_current)
        database_test_imputed[country] = pd.DataFrame(data = df_data,
                                                      columns = df_current.columns,
                                                      index = df_current.index)
        
    except Exception as e:
        print(country + " " + str(e))
    

Malaysia Shape of passed values is (25, 10), indices imply (25, 15)
Peru Shape of passed values is (25, 13), indices imply (25, 15)
Zambia Shape of passed values is (25, 11), indices imply (25, 15)
Bulgaria could not convert string to float: '--'
Netherlands could not convert string to float: '--'
Singapore Shape of passed values is (25, 13), indices imply (25, 15)
India Shape of passed values is (25, 14), indices imply (25, 15)
Paraguay Shape of passed values is (25, 10), indices imply (25, 15)
Morocco Shape of passed values is (25, 12), indices imply (25, 15)
Norway Shape of passed values is (25, 13), indices imply (25, 15)
Jamaica Shape of passed values is (25, 10), indices imply (25, 15)
Hong Kong SAR Shape of passed values is (25, 10), indices imply (25, 15)


## Preprocess Data

The different variables have different absolute vaules. Some machine learning models will have problems with this type of input. That is the reason I preprocess the data as follows:

First I want to have only growth rates. For a time series $ X = (x_{1}, ..., x_{n})$ the growth rate for $x_{i}$ is defined as:

\begin{equation}
\hat{x}_{i} = \frac{x_{i}}{x_{i-1}}
\end{equation}
    
The first value of the original serires will not be mapped. Another issue occoures when using only this transformation. I compare a 10 % increase and a 10 % decrease of a certain vaule. Let's denothe the increase with $\hat{x}_{I}$ and the decrease with $\hat{x}_{D}$. Now the following applies:

\begin{equation}
\hat{x}_{I} = 1.1 \\
\hat{x}_{D} = 0.9
\end{equation}
 
 so
 
 \begin{equation}
|\hat{x}_{I}| \neq |\hat{x}_{D}|.
\end{equation}

In order to obation the same absolute value for both directions, I will apply a logarithmic transformation:

\begin{equation}
\hat{x}_{i} = \ln(\frac{x_{i}}{x_{i-1}})
\end{equation}

To avoid arguments that are not defined for the logarithm I shift the fraction by $|\min_i(x_{i})|  + 0.001$:
 
\begin{equation}
\hat{x}_{i} = \ln(\frac{x_{i}}{x_{i-1}} + |\min_i(x_{i})| + 0.001)
\end{equation}

In same cases there is $x_{i} = x_{i-1} = 0$ which will cause an error because of divisin by zero. I add a some constant to avoid this problem.  

As there are no statistics from the dataset involved, this transformation can be appplied to the whole dataset. Therefore I first glue the training, validation and test dataset together:

In [42]:
def combine_datasets(database_training, database_validation, database_test):
    database_complete = {}

    for country in database_training.keys():
        database_complete[country] = database_training[country].append(database_validation[country]).append(database_test[country])
        
    return database_complete

database_imputed = combine_datasets(database_training_imputed, database_validation_imputed, database_test_imputed)

With this function I transform the whole dataframe

In [43]:
def convert_time_series_to_relative(df):
    # Assings each t the Values of ln(X_t / X_(t-1))
    # X_0 will be dropped
    
    df_new = df.iloc[1:, :].copy()
    
    for variable in df.columns:
        try:
            
            if not (df[variable] != 0).all():
                df[variable] = df[variable] + 0.001
                
            #Check if this tranformation is correct!
           # df_new[variable] = df[variable].iloc[:-1].values / df[variable].iloc[1:].values
            df_new[variable] = df[variable].iloc[1:].values / df[variable].iloc[:-1].values 
            
            df_new[variable] = np.log(df_new[variable] + 0.001 + np.abs(np.min(df_new[variable])))
        except Exception as e:
            print(country + " " + str(e))
        
    return df_new

and apply it to the database

In [44]:
database_transformed = {}

for country in database_imputed.keys():
    
    database_transformed[country] = convert_time_series_to_relative(database_imputed[country])
  

## Index Shift of the data

reference: paper crystal ball; shifting index for supervised learning, tuples look like (x_t-1, y_t)

In [45]:
def preprocess_for_supervised_learning(df):
     
    df_y = df.loc[:, 'y']
    df_variables = df.drop(['y'], axis=1)
    
    df_variables.index = df_variables.index - 1 
   
    df = pd.DataFrame(df_y).join(df_variables, how='inner')
    
    return df


df = database_transformed['Germany']
df_supervised = preprocess_for_supervised_learning(df)
df_supervised.head()

,y,GHG,Crude_oil_production,"Inflation, average consumer prices",Current account balance,ExchangeR,Population,Working_age_population,Fertility_rates,Unemployment rate,Patents_on_environment_technologies,PPP,TOTMAT,Material_consumption,General government net lending/borrowing
1981,1.981155,0.632298,0.616743,1.018168,1.354160,0.594607,0.683946,0.697905,0.624690,0.806413,0.633409,0.661480,0.656005,0.660662,0.368417
1982,-6.907755,0.660700,0.622615,0.940714,1.776416,0.582665,0.683105,0.697649,0.588342,0.717180,0.636903,0.663734,0.656005,0.660662,0.368417
1983,1.646981,0.668897,0.631374,0.980804,1.984612,0.616934,0.682684,0.696084,0.624690,0.610245,0.640212,0.661181,0.656005,0.660662,0.368417
1984,2.195223,0.660570,0.649201,1.032039,1.947378,0.572700,0.683299,0.693316,0.624690,0.617418,0.705977,0.664048,0.656005,0.660662,0.368417
1985,2.071950,0.660539,0.632125,0.629685,1.894802,0.389478,0.684657,0.691693,0.624690,0.593464,0.669449,0.674149,0.656005,0.660662,0.368417


In [46]:
df.head()

,y,GHG,Crude_oil_production,"Inflation, average consumer prices",Current account balance,ExchangeR,Population,Working_age_population,Fertility_rates,Unemployment rate,Patents_on_environment_technologies,PPP,TOTMAT,Material_consumption,General government net lending/borrowing
1981,1.981155,0.640923,0.621216,1.130759,1.708998,0.684085,0.685195,0.697405,0.590653,0.826003,0.644128,0.644167,0.656005,0.660662,0.368417
1982,-6.907755,0.632298,0.616743,1.018168,1.354160,0.594607,0.683946,0.697905,0.624690,0.806413,0.633409,0.661480,0.656005,0.660662,0.368417
1983,1.646981,0.660700,0.622615,0.940714,1.776416,0.582665,0.683105,0.697649,0.588342,0.717180,0.636903,0.663734,0.656005,0.660662,0.368417
1984,2.195223,0.668897,0.631374,0.980804,1.984612,0.616934,0.682684,0.696084,0.624690,0.610245,0.640212,0.661181,0.656005,0.660662,0.368417
1985,2.071950,0.660570,0.649201,1.032039,1.947378,0.572700,0.683299,0.693316,0.624690,0.617418,0.705977,0.664048,0.656005,0.660662,0.368417


Applying it to the whole database:

In [47]:
database_supervised = {}

for country in database_transformed.keys():
    
    database_supervised[country] = preprocess_for_supervised_learning(database_transformed[country])

Finally I split the dataset again to obtain the training, validation and test sets:

In [48]:
database_training, database_validation, database_test = split_dataset(database_transformed)
database_training_sv, database_validation_sv, database_test_sv = split_dataset(database_supervised)

## Standardization 

Finally, transform data so that every colun has 0 mean and sdtdev 1. This is especially important for neural networks.  

In [49]:
def standardization(df_training, df_validation, df_test):
     
    scaler = preprocessing.StandardScaler().fit(df_training)
    
    return scaler.transform(df_training), scaler.transform(df_validation), scaler.transform(df_test), scaler

Applying it to the whole dataset and keeping track of the scaler instance for each country:

In [50]:
database_training_sv_standard = {}
database_validation_sv_standard = {}
database_test_sv_standard = {}
database_scaler = {}

for country in database_training_sv.keys():
    
    tr, val, test, scaler = standardization(database_training_sv[country], database_validation_sv[country], database_test_sv[country])
    
    database_training_sv_standard[country] = pd.DataFrame(tr)
    database_validation_sv_standard[country] = pd.DataFrame(val)
    database_test_sv_standard[country]  = pd.DataFrame(test)
    database_scaler[country] = scaler
    
    

I also serialize the whole database, so I can start coding without rerunning the whole data handling.

In [51]:
database_dir = os.path.join(r'C:/Users/hauer/Documents/Repositories/cfds_project', 'database.pickle')
with open(database_dir, 'wb') as f:
    save = {
        'database_training': database_training,
        'database_validation': database_validation,
        'database_test': database_test,
        'database_training_sv': database_training_sv,
        'database_validation_sv': database_validation_sv,
        'database_test_sv': database_test_sv,
        
        'database_training_sv_standard': database_training_sv_standard,
        'database_validation_sv_standard': database_validation_sv_standard,
        'database_test_sv_standard': database_test_sv_standard,
        
        'database_scaler': database_scaler,
    }
    pickle.dump(save, f, pickle.HIGHEST_PROTOCOL)

In [52]:
with open(database_dir,'rb') as f: 
    db = pickle.load(f)
    
database_training = db['database_training']
database_validation = db['database_validation']
database_test = db['database_test']

database_training_sv = db['database_training_sv']
database_validation_sv = db['database_validation_sv']
database_test_sv = db['database_test_sv']

database_training_sv_standard = db['database_training_sv_standard']
database_validation_sv_standard = db['database_validation_sv_standard']
database_test_sv_standard = db['database_test_sv_standard']

database_scaler = db['database_scaler']

# Models

Loss function: MSE. To determine a to determine a good choise for the hyperparameter, a grid search is done for some of the models. 

List of all models I will use: 

In [53]:
models = []

For models that are in the sklearn framework with the `fit` and `predict` methods, I use this function to perform the evaulation: 

In [54]:
def forecast(model, df_training, df_prediction):
   
    X_train = df_training.iloc[:,1:].values
    y_train = df_training.iloc[:,0].values
    
    X_test = df_prediction.iloc[:,1:].values
    # y_test = df_prediction.iloc[:,0].values
    
    model.fit(X_train, y_train)
    
    y_predicted = model.predict(X_test)
  
    result = pd.Series(data=y_predicted, index = df_prediction.index)
    
    return result

In [55]:
country = 'France'

df_training = database_training[country]
df_validation = database_validation[country]
df_test = database_test[country]

df_training_sv = database_training_sv[country]
df_validation_sv = database_validation_sv[country]
df_test_sv = database_test_sv[country]


t_forecast_test = df_test_sv.index.values

start_forecast = df_validation_sv.index.values[0]
t_forecast_validation = df_validation_sv.index.values

In [56]:
#Check forecasting for supervised learning in the evaliation. values from 2011 forecast value for 2012 

print(start_forecast)
print(t_forecast_validation)
print(t_forecast_test)


2005
[2005 2006 2007 2008 2009 2010]
[2011 2012 2013 2014 2015 2016]


In [57]:
df_training_sv

,y,GHG,Crude_oil_production,"Inflation, average consumer prices",Current account balance,ExchangeR,Population,Working_age_population,Fertility_rates,Unemployment rate,Patents_on_environment_technologies,PPP,TOTMAT,Material_consumption,General government net lending/borrowing
1981,2.473053,0.609451,0.601519,-0.071137,2.015016,0.683670,0.697584,0.694394,0.665490,0.675237,0.586405,0.702128,0.653645,0.637485,0.592842
1982,2.610877,0.608494,0.616214,-0.195111,1.654946,0.658284,0.697287,0.693936,0.638064,0.584808,0.532961,0.701856,0.653645,0.637485,0.423063
1983,2.464965,0.607449,0.735256,-0.169405,1.608607,0.651544,0.697035,0.693557,0.665490,0.705702,0.628229,0.691001,0.642950,0.621882,0.535152
1984,2.524740,0.614790,0.752457,-0.232184,1.662042,0.587784,0.697066,0.691691,0.665490,0.646200,0.563710,0.685382,0.611977,0.648545,0.540708
1985,2.512819,0.614245,0.671075,-0.758447,-6.907755,0.433510,0.697091,0.690409,0.665490,0.641737,0.604574,0.689206,0.643062,0.643581,0.534545
1986,2.542200,0.622645,0.664090,0.283941,1.181913,0.494493,0.697216,0.690698,0.665490,0.647443,0.585682,0.674044,0.653645,0.637485,0.231850
1987,2.514784,0.622493,0.631039,-0.157608,1.755524,0.566952,0.697352,0.690644,0.665490,0.613022,0.632041,0.672481,0.653645,0.637485,0.643985
1988,2.574058,0.649718,0.593164,0.902202,1.760519,0.611326,0.697439,0.690219,0.665490,0.622512,0.568002,0.671008,0.653645,0.637485,0.282147
1989,2.500086,0.622645,0.573785,-2.499423,1.878428,0.485695,0.697248,0.689511,0.665490,0.612577,0.632082,0.668777,0.634775,0.649842,0.691569
1990,2.479959,0.666965,0.598276,2.358763,1.696108,0.592210,0.709726,0.688703,0.665490,0.644743,0.665755,0.670004,0.647338,0.647157,0.591789


In [58]:
df_validation_sv

,y,GHG,Crude_oil_production,"Inflation, average consumer prices",Current account balance,ExchangeR,Population,Working_age_population,Fertility_rates,Unemployment rate,Patents_on_environment_technologies,PPP,TOTMAT,Material_consumption,General government net lending/borrowing
2005,2.472775,0.613357,0.599137,0.034992,1.982254,0.566827,0.698170,0.690696,0.692185,0.627076,0.680650,0.662556,0.654913,0.645666,0.306906
2006,2.544771,0.612713,0.568724,-0.124593,1.486871,0.523548,0.697776,0.690326,0.665490,0.578999,0.696096,0.670667,0.646221,0.654759,0.536216
2007,2.506168,0.621832,0.611837,0.692215,2.455300,0.532774,0.697476,0.689533,0.665490,0.595478,0.640012,0.669637,0.665776,0.623750,0.625513
2008,2.431226,0.611656,0.568061,-2.734537,1.725731,0.602292,0.697257,0.689084,0.665490,0.740770,0.692717,0.662656,0.696727,0.577979,1.039990
2009,-6.907755,0.631547,0.609468,2.847283,1.786752,0.600992,0.697158,0.689151,0.665490,0.640827,0.634489,0.669343,0.674039,0.624392,0.464254
2010,2.359842,0.600388,0.610675,0.300099,1.823267,0.544789,0.697144,0.688140,0.665490,0.628213,0.647360,0.666520,0.649784,0.650087,0.321705


In [59]:
df_test_sv

,y,GHG,Crude_oil_production,"Inflation, average consumer prices",Current account balance,ExchangeR,Population,Working_age_population,Fertility_rates,Unemployment rate,Patents_on_environment_technologies,PPP,TOTMAT,Material_consumption,General government net lending/borrowing
2011,2.517120,0.642341,0.556448,-0.000240,1.783357,0.618015,0.697144,0.686970,0.665490,0.662850,0.646460,0.675906,0.670051,0.619865,0.468260
2012,2.434525,0.620855,0.601822,-0.733306,1.678011,0.553429,0.697264,0.686972,0.665490,0.660214,0.592711,0.654219,0.655266,0.636155,0.372171
2013,2.573217,0.575758,0.592631,-0.429025,1.902243,0.571851,0.698824,0.686839,0.665490,0.630214,0.600781,0.671563,0.664852,0.628741,0.461931
2014,2.559348,0.643561,0.659005,-1.714683,1.650836,0.677629,0.696501,0.687477,0.639453,0.635429,0.603573,0.674703,0.685897,0.609256,0.444213
2015,2.520265,0.631547,0.598197,1.230182,1.815972,0.573343,0.696023,0.688117,0.665490,0.614274,0.567547,0.656093,0.671279,0.623806,0.475034
2016,2.505648,0.643295,0.571897,1.346201,1.839840,0.560569,0.695804,0.688365,0.665490,0.597859,0.635782,0.665148,0.624845,0.678411,0.345922


## WOE

In [60]:
name = 'WOE'
y_forecast = get_predictions_weo(df_weo, country = country,
                                 start_forecast =  t_forecast_validation[0],
                                 end_forecast = t_forecast_validation[-1])
mse = mean_squared_error(y_forecast, df_validation_sv['y'].values)
models.append( (name, y_forecast, mse))

## OLS

The ordinary least squares regression is the most famous and basic model in econometrics. The depentend variable is shown as:

\begin{equation}
y = x_1\beta_1 + x_2\beta_2 + ... x_N\beta_N + \beta_{N+1}
\end{equation}

In [61]:
name = 'OLS'
model = LinearRegression()
y_forecast = forecast(model, df_training_sv, df_validation_sv)
y_forecast = y_forecast.values
mse = mean_squared_error(y_forecast, df_validation_sv['y'].values)
models.append( (name, y_forecast, mse))

## ARIMA

The autoregressive integrated moving average ARIMA($p$, $d$, $q$) model is used in time series analysis.

\begin{equation}
X_t - \alpha_1X_{t-1} - ... - \alpha_pX_{t-p} =  \epsilon_t + \theta_1\epsilon_{t-1} + ... + \theta_q\epsilon_{t-q}
\end{equation}

Here $\alpha _{i}$ are the parameters of the autoregressive part of the model, $\theta _{i}$ are the parameters of the moving average part, $d$ is the degree of differencing and $\epsilon _{t}$ are error terms. There is an implementaion in python in the pmdarima package, which automatically discovers the optimal order for an ARIMA model with exogenous variables. 

In [62]:
name = 'ARIMA'

y_train = df_training_sv.iloc[:, 0]
X_train = df_training_sv.iloc[:, 1:]
y_validation = df_validation_sv.iloc[:, 0]
X_validation = df_validation_sv.iloc[:, 1:]

model = auto_arima(y = y_train,
                   trace=True, 
                   start_p=0,
                   max_p=3,
                   start_q=0,
                   max_q=3,
                   seasonal = False,
                   stepwise= True,
                   exogenous=X_train) 

model.fit(y= y_train, exogenous=X_train)

y_forecast = model.predict(n_periods=y_validation.shape[0],
                      exogenous = X_validation)
mse = mean_squared_error(y_forecast, y_validation)
models.append( (name, y_forecast, mse))

C:\Users\hauer\anaconda3\envs\test\lib\site-packages\statsmodels\base\model.py:548: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)


Fit ARIMA: order=(0, 0, 0); AIC=-16.186, BIC=2.663, Fit time=0.095 seconds


C:\Users\hauer\anaconda3\envs\test\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


Fit ARIMA: order=(1, 0, 0); AIC=-14.909, BIC=5.118, Fit time=7.367 seconds


C:\Users\hauer\anaconda3\envs\test\lib\site-packages\statsmodels\tsa\tsatools.py:716: RuntimeWarning: divide by zero encountered in log
  invmacoefs = -np.log((1-macoefs)/(1+macoefs))
C:\Users\hauer\anaconda3\envs\test\lib\site-packages\statsmodels\tools\numdiff.py:243: RuntimeWarning: invalid value encountered in subtract
  **kwargs)).imag/2./hess[i, j]
C:\Users\hauer\anaconda3\envs\test\lib\site-packages\statsmodels\tsa\tsatools.py:689: RuntimeWarning: invalid value encountered in true_divide
  newparams = ((1-np.exp(-params))/(1+np.exp(-params))).copy()
C:\Users\hauer\anaconda3\envs\test\lib\site-packages\statsmodels\tsa\tsatools.py:690: RuntimeWarning: invalid value encountered in true_divide
  tmp = ((1-np.exp(-params))/(1+np.exp(-params))).copy()
C:\Users\hauer\anaconda3\envs\test\lib\site-packages\statsmodels\tsa\kalmanf\kalmanfilter.py:224: RuntimeWarning: invalid value encountered in log
  Z_mat.astype(complex), R_mat, T_mat)
C:\Users\hauer\anaconda3\envs\test\lib\site-package

...

C:\Users\hauer\anaconda3\envs\test\lib\site-packages\statsmodels\base\model.py:548: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
C:\Users\hauer\anaconda3\envs\test\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


Fit ARIMA: order=(1, 0, 3); AIC=-29.248, BIC=-5.687, Fit time=9.639 seconds


C:\Users\hauer\anaconda3\envs\test\lib\site-packages\statsmodels\base\model.py:548: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
C:\Users\hauer\anaconda3\envs\test\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


Fit ARIMA: order=(0, 0, 3); AIC=-31.335, BIC=-8.952, Fit time=8.367 seconds
Total fit time: 42.818 seconds


C:\Users\hauer\anaconda3\envs\test\lib\site-packages\statsmodels\base\model.py:548: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
C:\Users\hauer\anaconda3\envs\test\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


## Elastic Net 

In [63]:

parameters = {'alpha': [0.1, 0.25, 1.0, 2.0, 4.0, 8.0, 16, 32, 128]}
model = Lasso()

clf = GridSearchCV(model, parameters)

X_train = df_training_sv.iloc[:,1:].values
y_train = df_training_sv.iloc[:,0].values


clf.fit(X_train, y_train)

print(clf.best_params_)

{'alpha': 0.1}


In [64]:
name = 'LASSO'
model = Lasso(alpha = 0.1)
y_forecast = forecast(model, df_training_sv, df_validation_sv)
y_forecast = y_forecast.values
mse = mean_squared_error(y_forecast, df_validation_sv['y'].values)
models.append( (name, y_forecast, mse))

## Support Vector Regression

In [65]:
parameters = {'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'C': [0.1, 0.25, 1.0, 2.0, 4.0, 8.0],
             'epsilon': [0.01, 0.05, 0.1, 0.25, 0.5]}
model = SVR()
clf = GridSearchCV(model, parameters)

X_train = df_training_sv.iloc[:,1:].values
y_train = df_training_sv.iloc[:,0].values


clf.fit(X_train, y_train)

print(clf.best_params_)

{'C': 0.1, 'epsilon': 0.05, 'kernel': 'sigmoid'}


In [66]:
name = 'SVR'
model = SVR(C=0.1, epsilon=0.05, kernel = 'sigmoid')
y_forecast = forecast(model, df_training_sv, df_validation_sv)
y_forecast = y_forecast.values
mse = mean_squared_error(y_forecast, df_validation_sv['y'].values)
models.append( (name, y_forecast, mse))

## Regression Tree

In [67]:
parameters = {'max_depth': [2,3,4,5,10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 5, 10],
     'min_impurity_decrease': [0.0001, 0.001, 0.01, 0.01] }
model = DecisionTreeRegressor()
clf = GridSearchCV(model, parameters)

X_train = df_training_sv.iloc[:,1:].values
y_train = df_training_sv.iloc[:,0].values


clf.fit(X_train, y_train)

print(clf.best_params_)

{'max_depth': 2, 'min_impurity_decrease': 0.0001, 'min_samples_leaf': 10, 'min_samples_split': 2}


In [68]:
name = 'Tree'
model = DecisionTreeRegressor(max_depth = 2,
                              min_impurity_decrease =  0.0001, 
                              min_samples_leaf = 10,
                              min_samples_split =  2)
y_forecast = forecast(model, df_training_sv, df_validation_sv)
y_forecast = y_forecast.values
mse = mean_squared_error(y_forecast, df_validation_sv['y'].values)
models.append( (name, y_forecast, mse))

## GBM 

In [69]:
parameters = {'n_estimators':[5, 10, 20, 50, 100], 
              'max_depth':[1, 2, 4],
             'learning_rate': [0.01, 0.03, 0.1, 0.25],
             'min_samples_split':[2, 10]}
model = GradientBoostingRegressor()
clf = GridSearchCV(model, parameters)

X_train = df_training_sv.iloc[:,1:].values
y_train = df_training_sv.iloc[:,0].values


clf.fit(X_train, y_train)

GridSearchCV(cv=None, error_score=nan,
             estimator=GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0,
                                                 criterion='friedman_mse',
                                                 init=None, learning_rate=0.1,
                                                 loss='ls', max_depth=3,
                                                 max_features=None,
                                                 max_leaf_nodes=None,
                                                 min_impurity_decrease=0.0,
                                                 min_impurity_split=None,
                                                 min_samples_leaf=1,
                                                 min_samples_split=2,
                                                 min_weight_fraction_leaf=0.0,
                                                 n_estimators=100,
                                                 n_ite...
                            

In [70]:
print(clf.best_params_)

{'learning_rate': 0.01, 'max_depth': 2, 'min_samples_split': 2, 'n_estimators': 5}


In [71]:
name = 'GBM'
model = GradientBoostingRegressor(n_estimators = 5, max_depth = 2, 
                                  min_samples_split=2, learning_rate = 0.01)
y_forecast = forecast(model, df_training_sv, df_validation_sv)
y_forecast = y_forecast.values
mse = mean_squared_error(y_forecast, df_validation_sv['y'].values)
models.append( (name, y_forecast, mse))


## RNN

see Notebook 2.

# Running the models

This is the final evaluation, so models will be trained on the training set combined with the test set and the MSE is calculated with the test set. 

## Training on every country on its own

In [72]:
y_forecast_WEO = {}
y_forecast_ARIMA = {}
y_forecast_OLS = {}
y_forecast_LASSO = {}
y_forecast_SVR = {}
y_forecast_TREE = {}
y_forecast_GBM = {}

df_result_single_countries = pd.DataFrame(columns=['country', 'WEO', 'ARIMA', 'OLS', 'LASSO', 'SVR', 'TREE', 'GBM'])

for country in database_training_sv.keys():

    df_training_sv = database_training_sv[country]
    df_validation_sv = database_validation_sv[country]
    
    df_training_sv = pd.concat([df_training_sv, df_validation_sv])
    
    df_test_sv = database_test_sv[country]
    
    t_forecast_test = df_test_sv.index.values
    start_forecast = df_test_sv.index.values[0]

    
    # WEO
    
    y_forecast = get_predictions_weo(df_weo, country = country,
                                     start_forecast =  t_forecast_test[0],
                                     end_forecast = t_forecast_test[-1])
    mse_WEO = mean_squared_error(y_forecast, df_test_sv['y'].values)
    y_forecast_WEO[country] = y_forecast
    
   
    
    try:
    
        y_train = df_training_sv.iloc[:, 0]
        X_train = df_training_sv.iloc[:, 1:]
        y_validation = df_test_sv.iloc[:, 0]
        X_validation = df_test_sv.iloc[:, 1:]

        model = auto_arima(y = y_train,
                           trace=True, 
                           start_p=0,
                           max_p=3,
                           start_q=0,
                           max_q=3,
                           seasonal = False,
                           stepwise= True,
                           exogenous=X_train) 

        model.fit(y= y_train, exogenous=X_train)

        y_forecast = model.predict(n_periods=y_validation.shape[0],
                              exogenous = X_validation)
        y_forecast_ARIMA[country] = y_forecast
        mse_ARIMA = mean_squared_error(y_forecast, y_validation)
        
    except Exception as e:
        mse_ARIMA = -1
        y_forecast_ARIMA[country] = [0 for i in range(len(y_forecast))]
        

    
    # OLS
    
    model = LinearRegression()
    y_forecast = forecast(model, df_training_sv, df_test_sv)
    y_forecast = y_forecast.values
    mse_OLS = mean_squared_error(y_forecast, df_test_sv['y'].values)
    y_forecast_OLS[country] = y_forecast
    
    # LASSO

    model = Lasso(alpha = 0.25)
    y_forecast = forecast(model, df_training_sv, df_test_sv)
    y_forecast = y_forecast.values
    mse_LASSO = mean_squared_error(y_forecast, df_test_sv['y'].values)
    y_forecast_LASSO[country] = y_forecast
    
    
    # SVR
    
    model = SVR(C=0.1, epsilon=0.05, kernel = 'sigmoid')
    y_forecast = forecast(model, df_training_sv, df_test_sv)
    y_forecast = y_forecast.values
    mse_SVR = mean_squared_error(y_forecast, df_test_sv['y'].values)
    y_forecast_SVR[country] = y_forecast
    
    
    # Regression Tree
    
    model = DecisionTreeRegressor(max_depth = 2,
                              min_impurity_decrease =  0.01, 
                              min_samples_leaf = 2,
                              min_samples_split =  10)
    y_forecast = forecast(model, df_training_sv, df_test_sv)
    y_forecast = y_forecast.values
    mse_TREE = mean_squared_error(y_forecast, df_test_sv['y'].values)
    y_forecast_TREE[country] = y_forecast


    
    # GBM
    
    model = GradientBoostingRegressor(n_estimators = 5, max_depth = 2, 
                                      min_samples_split=10, learning_rate = 0.01)
    y_forecast = forecast(model, df_training_sv, df_test_sv)
    y_forecast = y_forecast.values
    mse_GBM = mean_squared_error(y_forecast, df_test_sv['y'].values)
    y_forecast_GBM[country] = y_forecast

    
    
    
    df_result_single_countries = pd.concat([pd.DataFrame([[country ,mse_WEO, mse_ARIMA, mse_OLS, mse_LASSO, mse_SVR, mse_TREE, mse_GBM]],
                                                         columns=df_result_single_countries.columns),
                                            df_result_single_countries],ignore_index=True)
    

C:\Users\hauer\anaconda3\envs\test\lib\site-packages\statsmodels\base\model.py:548: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)


Fit ARIMA: order=(0, 0, 0); AIC=131.561, BIC=153.980, Fit time=0.215 seconds


C:\Users\hauer\anaconda3\envs\test\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


Fit ARIMA: order=(1, 0, 0); AIC=133.527, BIC=157.347, Fit time=7.278 seconds


C:\Users\hauer\anaconda3\envs\test\lib\site-packages\statsmodels\base\model.py:548: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
C:\Users\hauer\anaconda3\envs\test\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


...

C:\Users\hauer\anaconda3\envs\test\lib\site-packages\statsmodels\base\model.py:548: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
C:\Users\hauer\anaconda3\envs\test\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


Fit ARIMA: order=(1, 0, 2); AIC=-2.124, BIC=24.499, Fit time=10.915 seconds


C:\Users\hauer\anaconda3\envs\test\lib\site-packages\statsmodels\base\model.py:548: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
C:\Users\hauer\anaconda3\envs\test\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


Fit ARIMA: order=(3, 0, 3); AIC=-0.353, BIC=30.473, Fit time=13.807 seconds
Total fit time: 96.240 seconds


C:\Users\hauer\anaconda3\envs\test\lib\site-packages\statsmodels\base\model.py:548: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
C:\Users\hauer\anaconda3\envs\test\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


In [73]:
classic_models = ['WEO', 'OLS', 'ARIMA']
machine_learning_models = [x for x in df_result_single_countries.keys() if x not in ['country', 'WEO', 'OLS', 'ARIMA']]

Evaluation for single models

In [74]:
result_single_models = {}


for machine_learning_model in machine_learning_models:
    
    result = []

    for i in range(0,len(df_result_single_countries)):
    
        df_current = df_result_single_countries.iloc[i,:]
       
        for classic_model in classic_models:
            
            if df_current[classic_model] <  df_current[machine_learning_model]:
                x = 0
            else:
                x = 1
                
            result.append(x)
   
    result_single_models[machine_learning_model] = result       

Evaluation on aggregated level

In [75]:
result_single = []

for i in range(0,len(df_result_single_countries)):
    
    df_current = df_result_single_countries.iloc[i,:]
    
    for classic_model in classic_models:
        for machine_learning_model in machine_learning_models:
            if df_current[classic_model] <  df_current[machine_learning_model]:
                x = 0
            else:
                x = 1
            result_single.append(x)

## Training the whole data set

In [76]:
df_training_sv_complete = pd.DataFrame()
for country in database_training_sv.keys():
    df_training_sv = database_training_sv[country]
    df_validation_sv = database_validation_sv[country]
    
    df_training_sv = pd.concat([df_training_sv, df_validation_sv])

    df_training_sv_complete = pd.concat([df_training_sv_complete, df_training_sv]) 

Defining RNN

In [77]:
class LSTMNet(nn.Module):
    def __init__(self, input_size, seq_len, output_size, hidden_dim, n_layers):
        super(LSTMNet, self).__init__()

        self.hidden_dim = hidden_dim
        self.seq_len = seq_len
               
        
        self.lstm1 = nn.LSTM(input_size, hidden_dim)
        self.lstm2 = nn.LSTM(hidden_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_size)

    def forward(self, x, hidden_1, state_1, hidden_2, state_2):
        r_out, (hidden_out, state_out) = self.lstm1(x, (hidden_1, state_1))      
        r_out, (hidden_out, state_out) = self.lstm2(r_out, (hidden_2, state_2))
        r_out = self.fc(r_out)
        
        return r_out
        
    def initHidden(self):
        return zeros(1, self.seq_len, self.hidden_dim)

In [78]:
y_forecast_WEO = {}
y_forecast_OLS = {}
y_forecast_LASSO = {}
y_forecast_SVR = {}
y_forecast_TREE = {}
y_forecast_GBM = {}
y_forecast_RNN = {}

df_result_all_countries = pd.DataFrame(columns=['country', 'WEO', 'OLS', 'LASSO', 'SVR', 'TREE', 'GBM', 'RNN'])
N, dummy_dim = database_training_sv_standard['Germany'].shape
dummy_dim -= 1


for country in database_training_sv.keys():
#for country in ['Germany', 'France']:

    
    df_test_sv = database_test_sv[country]
    
    t_forecast_test = df_test_sv.index.values
    start_forecast = df_test_sv.index.values[0]

    
    # WEO
    
    y_forecast = get_predictions_weo(df_weo, country = country,
                                     start_forecast =  t_forecast_test[0],
                                     end_forecast = t_forecast_test[-1])
    mse_WEO = mean_squared_error(y_forecast, df_test_sv['y'].values)
    y_forecast_WEO[country] = y_forecast
    
     

    
    # OLS
    
    model = LinearRegression()
    y_forecast = forecast(model, df_training_sv_complete, df_test_sv)
    y_forecast = y_forecast.values
    mse_OLS = mean_squared_error(y_forecast, df_test_sv['y'].values)
    y_forecast_OLS[country] = y_forecast
    
    
     # LASSO

    model = Lasso(alpha = 0.25)
    y_forecast = forecast(model, df_training_sv_complete, df_test_sv)
    y_forecast = y_forecast.values
    mse_LASSO = mean_squared_error(y_forecast, df_test_sv['y'].values)
    y_forecast_LASSO[country] = y_forecast
    
    
    # SVR
    
    model = SVR(C=0.1, epsilon=0.05, kernel = 'sigmoid')
    y_forecast = forecast(model, df_training_sv_complete, df_test_sv)
    y_forecast = y_forecast.values
    mse_SVR = mean_squared_error(y_forecast, df_test_sv['y'].values)
    y_forecast_SVR[country] = y_forecast
    
    
    # Regression Tree
    
    model = DecisionTreeRegressor(max_depth = 2,
                              min_impurity_decrease =  0.01, 
                              min_samples_leaf = 2,
                              min_samples_split =  10)
    y_forecast = forecast(model, df_training_sv_complete, df_test_sv)
    y_forecast = y_forecast.values
    mse_TREE = mean_squared_error(y_forecast, df_test_sv['y'].values)
    y_forecast_TREE[country] = y_forecast

    
    # GBM
    
    name = 'GBM'
    model = GradientBoostingRegressor(n_estimators = 5, max_depth = 2, 
                                      min_samples_split=10, learning_rate = 0.01)
    y_forecast = forecast(model, df_training_sv_complete, df_test_sv)
    y_forecast = y_forecast.values
    mse_GBM = mean_squared_error(y_forecast, df_test_sv['y'].values)
    y_forecast_GBM[country] = y_forecast
    
    
    # RNN
    
    wdir= r'C:/Users/hauer/Documents/Repositories/cfds_project'
    save_dir = os.path.join(wdir, 'pytorch_models')
    model_name = 'rnn.torch'
    
    hidden_dim = 64
    model = LSTMNet(input_size=14, seq_len=17, output_size=1, hidden_dim=hidden_dim, n_layers=1)
    model.load_state_dict(load( os.path.join(save_dir, model_name)))
    
    
    df = database_training_sv_standard[country].append(database_validation_sv_standard[country]).append(database_test_sv_standard[country])

    n_forecast_validation, _ = database_test_sv_standard[country].shape

    X_eval = df.iloc[:,1:].values
    y_eval = df.iloc[:,0].values
    X_eval_T = from_numpy(X_eval).float()
    N, _ = X_eval_T.shape
    X_eval_T = X_eval_T.view([-1, N, dummy_dim])

    hidden_1 = zeros(1, N, hidden_dim)
    state_1 = zeros(1, N, hidden_dim)

    hidden_2 = zeros(1, N, hidden_dim)
    state_2 = zeros(1, N, hidden_dim)

    model.eval()
    with no_grad():
        y_hat = model(X_eval_T, hidden_1, state_1, hidden_2, state_2)

    y_hat =  y_hat.view(-1).numpy()
    y_forecast = y_hat[-n_forecast_validation:]
    
    # Inverse tranformation 
    
    scaler = database_scaler[country]
    df_output = database_test_sv_standard[country]
    df_output.iloc[:,0] = y_forecast
    df_output = pd.DataFrame(scaler.inverse_transform(df_output))
    y_forecast = df_output.iloc[:,0].values
    
    
    mse_RNN = mean_squared_error(y_forecast, df_test_sv['y'].values)

    y_forecast_RNN[country] = y_forecast
    

    
    
    
    df_result_all_countries = pd.concat([pd.DataFrame([[country ,mse_WEO, mse_OLS, mse_LASSO, mse_SVR, mse_TREE, mse_GBM, mse_RNN]],
                                                        columns=df_result_all_countries.columns),
                                         df_result_all_countries],ignore_index=True)
    

In [79]:
classic_models = ['WEO', 'OLS']
machine_learning_models = [x for x in df_result_all_countries.keys() if x not in ['country', 'WEO', 'OLS', 'ARIMA']]

Evaluation for single models

In [80]:
result_all_models = {}


for machine_learning_model in machine_learning_models:
    
    result = []

    for i in range(0,len(df_result_all_countries)):
    
        df_current = df_result_all_countries.iloc[i,:]
       
        for classic_model in classic_models:
            
            if df_current[classic_model] <  df_current[machine_learning_model]:
                x = 0
            else:
                x = 1
                
            result.append(x)
   
    result_all_models[machine_learning_model] = result 

Evaluation on aggregated level

In [81]:
result_all = []

for i in range(0,len(df_result_all_countries)):
    
    df_current = df_result_all_countries.iloc[i,:]
    
    for classic_model in classic_models:
        for machine_learning_model in machine_learning_models:
            if df_current[classic_model] <  df_current[machine_learning_model]:
                x = 0
            else:
                x = 1
            result_all.append(x)

# Final Analyis for statistical significance

Drawing $B$ bootstrap samples 

In [82]:
B = 1000

Models trained on single country

In [83]:
df_result_boot_models_single = pd.DataFrame(columns=['means', 'model'])

for model in result_single_models.keys():
    
    bootstrap = np.random.choice(result_single_models[model], (len(result_single_models[model]),B), replace = True)
    bootstrap_means = np.mean(bootstrap, axis = 1)
    
    model_name = [model for i in range(len(bootstrap_means))]
    
    df_result_boot_models_single = pd.concat([pd.DataFrame(list(zip(bootstrap_means, model_name)),
                                                    columns=df_result_boot_models_single.columns),
                                     df_result_boot_models_single],ignore_index=True)

In [84]:
ax = sns.boxplot(x='model', y='means', data=df_result_boot_models_single,
                width = 0.5)

ax.axhline(y=0.5, color='red',linestyle='--', alpha=0.75)

axes = plt.gca()
axes.set_ylim([0,1])

#plt.title("final test for statistical significance")


wdir= r'C:\Users\hauer\Documents\Repositories\cfds_project\plot_result'
save_dir = os.path.join(wdir, 'result_single_models.png')

plt.savefig(save_dir, dpi = 500, bbox_inches='tight')
plt.close()

Models trained on all countries

In [85]:
df_result_boot_models_single = pd.DataFrame(columns=['means', 'model'])

for model in result_all_models.keys():
    
    bootstrap = np.random.choice(result_all_models[model], (len(result_all_models[model]),B), replace = True)
    bootstrap_means = np.mean(bootstrap, axis = 1)
    
    model_name = [model for i in range(len(bootstrap_means))]
    
    df_result_boot_models_single = pd.concat([pd.DataFrame(list(zip(bootstrap_means, model_name)),
                                                    columns=df_result_boot_models_single.columns),
                                     df_result_boot_models_single],ignore_index=True)

In [86]:
ax = sns.boxplot(x='model', y='means', data=df_result_boot_models_single,
                width = 0.5)

ax.axhline(y=0.5, color='red',linestyle='--', alpha=0.75)

axes = plt.gca()
axes.set_ylim([0,1])

#plt.title("final test for statistical significance")


wdir= r'C:\Users\hauer\Documents\Repositories\cfds_project\plot_result'
save_dir = os.path.join(wdir, 'result_all_models.png')

plt.savefig(save_dir, dpi = 500, bbox_inches='tight')
plt.close()

Aggregated

In [87]:
bootstrap = np.random.choice(result_single, (len(result_single),B), replace = True)
bootstrap_means_single = np.mean(bootstrap, axis = 1)
approch_single = ['single' for i in range(len(bootstrap_means_single))]

bootstrap = np.random.choice(result_all, (len(result_all),B), replace = True)
bootstrap_means_all = np.mean(bootstrap, axis = 1)
approch_all = ['all' for i in range(len(bootstrap_means_all))]


bootstrap_means = np.concatenate( (bootstrap_means_all, bootstrap_means_single))
approach = approch_all + approch_single


In [88]:
df_result = pd.DataFrame({'means': bootstrap_means, 'approach':approach})

ax = sns.boxplot(x='approach', y='means', data=df_result,
                width = 0.5)

ax.axhline(y=0.5, color='red',linestyle='--', alpha=0.75)

axes = plt.gca()
axes.set_ylim([0,1])

#plt.title("final test for statistical significance")


wdir= r'C:\Users\hauer\Documents\Repositories\cfds_project\plot_result'
save_dir = os.path.join(wdir, 'result.png')

plt.savefig(save_dir, dpi = 500,bbox_inches='tight')
plt.close()